god damn model is start


In [82]:
import pandas as pd
import numpy as np
import xgboost as xgb
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import mean_squared_error,r2_score
from sklearn import model_selection

In [83]:
df = pd.read_csv('cleanedManuel_data.csv')

In [84]:
print(df.columns)

Index(['location', 'districk', 'neighborhood', 'room', 'living_room', 'area',
       'age', 'floor', 'price'],
      dtype='object')


In [85]:
X = df.drop(columns=['price'])
y = df['price']

In [86]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [87]:
#grid search
params = {"colsample_bytree":[0.4,0.5,0.6],
         "learning_rate":[0.01,0.02,0.09],
         "max_depth":[2,3,4,5,6],
         "n_estimators":[100,200,500,2000]}

In [88]:

xgb = XGBRegressor()


In [89]:
grid = GridSearchCV(xgb, params, cv = 10, n_jobs = -1, verbose = 2)


In [90]:
df['location'] = df['location'].astype('category')
df['districk'] = df['districk'].astype('category')
df['neighborhood'] = df['neighborhood'].astype('category')
df['room'] = df['room'].astype('int')
df['living_room'] = df['living_room'].astype('int')
df['area'] = df['area'].astype('int')
df['age'] = df['age'].astype('int')
df['floor'] = df['floor'].astype('int')
df['price'] = df['price'].astype('int')



In [91]:
print(X_train.dtypes)
print(y_train.dtypes)
print(X_train.isnull().sum())
print(y_train.isnull().sum())

location        object
districk        object
neighborhood    object
room             int64
living_room      int64
area             int64
age              int64
floor            int64
dtype: object
int64
location        0
districk        0
neighborhood    0
room            0
living_room     0
area            0
age             0
floor           0
dtype: int64
0


In [92]:
X_train = X_train.apply(lambda col: col.cat.codes if col.dtype.name == 'category' else col)


In [93]:
# objeler
object_cols = X_train.select_dtypes(include=['object']).columns
print("Object sütunlar:", object_cols)

# Bu sütunları one-hot encoding ile sayısal hale getir
X_train_encoded = pd.get_dummies(X_train, drop_first=True)


Object sütunlar: Index(['location', 'districk', 'neighborhood'], dtype='object')


In [94]:
X_test_encoded = pd.get_dummies(X_test, drop_first=True)

# Train ve test verilerini aynı sütun yapısına hizala
X_train_encoded, X_test_encoded = X_train_encoded.align(X_test_encoded, join='left', axis=1, fill_value=0)


In [28]:
grid.fit(X_train_encoded, y_train)


Fitting 10 folds for each of 180 candidates, totalling 1800 fits


GridSearchCV(cv=10,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None,...
                                    max_cat_to_onehot=None, max_delta_step=None,
                                    max_depth=None, max_leaves=None,
                                    min_child_weight=None, missing=nan,
                                    monotone_constraints=None,
                                    multi_strategy=None, n_estimators=None,
                                    n_jobs=None, num_parallel_tree=None,
                                    random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.4, 0.5, 0.6],
                         'learning_rate': [0.01, 0.02, 0.09],
                         'max_depth': [2, 3, 4, 5, 6],
                         'n_estimators': [100, 200, 500, 2000]},
             verbose=2)

In [95]:
grid.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [96]:
#grid search
params = {"colsample_bytree":[0.1,0.2,0.4,0.5],
         "learning_rate":[0.1,1,5,10],
         "max_depth":[2,3,4,5,6],
         "n_estimators":[500,2000,5000,10000]}

In [97]:
grid = GridSearchCV(xgb, params, cv = 10, n_jobs = -1, verbose = 2)


In [98]:
grid.fit(X_train_encoded, y_train)

Fitting 10 folds for each of 320 candidates, totalling 3200 fits


KeyboardInterrupt: 

In [34]:
grid.best_params_

{'colsample_bytree': 0.4,
 'learning_rate': 0.1,
 'max_depth': 4,
 'n_estimators': 2000}

In [99]:
##en uygun parametreler
xgb1 = XGBRegressor(colsample_bytree=0.4, learning_rate=0.09, max_depth=4, n_estimators=2000)

In [100]:
#model eğitimi
model_xgb = xgb1.fit(X_train_encoded, y_train)

In [101]:
model_xgb.predict(X_test_encoded[15:21])

array([48061.86 , 29843.412, 18677.432, 33508.203, 16836.834, 43202.613],
      dtype=float32)

In [102]:
y_test[15:21]

6653    35000
2465    23000
4615    16000
3151    42000
132     16750
2206    20000
Name: price, dtype: int64

In [103]:
model_xgb.score(X_test_encoded, y_test)

0.7126231940747272

In [104]:
model_xgb.score(X_train_encoded, y_train)

0.8769673335041887

In [76]:
print(model_xgb.predict(X_test_encoded))

[28369.488 21155.783 12271.943 ... 84682.39  40057.918 23761.814]
